In [95]:
import numpy as np
import os
import PIL
from PIL import Image
import tensorflow as tf
import matplotlib.pyplot as plt

# Training

In [96]:
batch_size = 128
img_height = 44
img_width = 33

In [97]:
data_dir = '/home/andy/Documents/Test_numbers'

In [98]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 1961 files belonging to 10 classes.
Using 1569 files for training.


In [99]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 1961 files belonging to 10 classes.
Using 392 files for validation.


In [100]:
class_names = train_ds.class_names
print(class_names)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


In [101]:
# normalization_layer = tf.keras.layers.Rescaling(1./255)
# normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
# image_batch, labels_batch = next(iter(normalized_ds))

In [102]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [103]:
num_classes = 10
def create_model():
    model = tf.keras.Sequential([
      tf.keras.layers.Rescaling(1./255),
      tf.keras.layers.Conv2D(32, 3, activation='relu'),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Conv2D(32, 3, activation='relu'),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Conv2D(32, 3, activation='relu'),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(num_classes)
    ])
    return model

In [104]:
model = create_model()
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [105]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=100
)

Epoch 1/100
13/13 [==============================] - 0s 12ms/step - loss: 2.0803 - accuracy: 0.4493 - val_loss: 1.5853 - val_accuracy: 0.9413
Epoch 2/100
13/13 [==============================] - 0s 5ms/step - loss: 0.9386 - accuracy: 0.9446 - val_loss: 0.2770 - val_accuracy: 0.9643
Epoch 3/100
13/13 [==============================] - 0s 5ms/step - loss: 0.1973 - accuracy: 0.9611 - val_loss: 0.1283 - val_accuracy: 0.9566
Epoch 4/100
13/13 [==============================] - 0s 5ms/step - loss: 0.1401 - accuracy: 0.9637 - val_loss: 0.0987 - val_accuracy: 0.9694
Epoch 5/100
13/13 [==============================] - 0s 5ms/step - loss: 0.0950 - accuracy: 0.9726 - val_loss: 0.0685 - val_accuracy: 0.9847
Epoch 6/100
13/13 [==============================] - 0s 5ms/step - loss: 0.0623 - accuracy: 0.9809 - val_loss: 0.0472 - val_accuracy: 0.9949
Epoch 7/100
13/13 [==============================] - 0s 5ms/step - loss: 0.0409 - accuracy: 0.9917 - val_loss: 0.0296 - val_accuracy: 1.0000
Epoch 8/100


Epoch 57/100
13/13 [==============================] - 0s 5ms/step - loss: 6.0033e-05 - accuracy: 1.0000 - val_loss: 9.3188e-05 - val_accuracy: 1.0000
Epoch 58/100
13/13 [==============================] - 0s 5ms/step - loss: 5.7674e-05 - accuracy: 1.0000 - val_loss: 8.9792e-05 - val_accuracy: 1.0000
Epoch 59/100
13/13 [==============================] - 0s 5ms/step - loss: 5.5410e-05 - accuracy: 1.0000 - val_loss: 8.6619e-05 - val_accuracy: 1.0000
Epoch 60/100
13/13 [==============================] - 0s 5ms/step - loss: 5.3266e-05 - accuracy: 1.0000 - val_loss: 8.3371e-05 - val_accuracy: 1.0000
Epoch 61/100
13/13 [==============================] - 0s 5ms/step - loss: 5.1281e-05 - accuracy: 1.0000 - val_loss: 8.0542e-05 - val_accuracy: 1.0000
Epoch 62/100
13/13 [==============================] - 0s 5ms/step - loss: 4.9319e-05 - accuracy: 1.0000 - val_loss: 7.7854e-05 - val_accuracy: 1.0000
Epoch 63/100
13/13 [==============================] - 0s 5ms/step - loss: 4.7561e-05 - accuracy: 1.0

In [106]:
path_to_numbers = '/home/andy/Documents/Test_numbers_NOLABEL'
import shutil

for i in range(5000, 5099):
    filename = os.path.join(path_to_numbers,'frame_{}_label_NO_LABEL.png'.format(i))
    img = Image.open(filename)
    image_array = np.asarray(img)
    a = model.predict(np.expand_dims(image_array, axis=0))
    val = np.argmax(a)
    shutil.copyfile(filename, os.path.join(path_to_numbers,'results', 'frame_{}_label_{}.png'.format(i,val)))

In [107]:
model.save_weights('pretrained_num_classification')

In [117]:
model.save('pretrained_model')

2022-07-11 16:12:27.820650: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: pretrained_model/assets


In [116]:
load_model = create_model()
load_model.load_weights('pretrained_num_classification')

path_to_numbers = '/home/andy/Documents/Test_numbers_NOLABEL'

filename = os.path.join(path_to_numbers,'frame_5035_label_NO_LABEL.png')
img = Image.open(filename)
image_array = np.asarray(img)

a = load_model.predict(np.expand_dims(image_array, axis=0))
val = np.argmax(a)
print(val)
# shutil.copyfile(filename, os.path.join(path_to_numbers,'results', 'frame_{}_label_{}.png'.format(i,val)))


7
